In [ ]:
import os
import cv2
from cv2 import aruco
import numpy as np
import matplotlib.pyplot as plt
import glob
import pandas as pd
import matplotlib.patches as patches
%matplotlib inline 

# This module is used to extract useful frames from a calibration video

In [ ]:
def select_frames(video_file, skip_frames=100, start_frame=0):
    """
    Allows the user to manually select frames from a video file for inspection and saving.

    Args:
        video_file (str): The path to the video file.
        skip_frames (int, optional): The number of frames to skip between selections. Defaults to 100.
        start_frame (int, optional): The starting frame index. Defaults to 0.

    Returns:
        None

    Displays each frame of the video file, allowing the user to inspect and save frames.
    Checkerboard patterns should have the entire board visible, without occlusions. 
    Aruco or Charuco boards should be in focus as possible. 
    Only full markers will be detected, but only a partial pattern need be visible.

    Controls:
        - Press 'S' to save the current frame.
        - Press 'A' to skip to the next frame.
        - Press 'Q' to quit the selection process.

    Saved frames will be stored in the 'Frames' directory alongside the video file.

    Note:
        This function uses OpenCV for video capture and frame manipulation.
    """
    
    cap = cv2.VideoCapture(video_file)
    if not cap.isOpened():
        print("Error: Could not open video file.")
        return
    
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    current_frame = start_frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    print("Please inspect each frame. Checkerboard patterns should have the entire board visible, without occlusions. Aruco or Charuco boards should be in focus as possible. Only full markers will be detected, but only a partial pattern need be visible.")
    print("Press 'S' to save the frame, 'A' to try the next frame, or 'Q' to quit")

    while current_frame<frame_count:
        ret, frame = cap.read()
        if not ret:
            break
        
        cv2.namedWindow('Frame', cv2.WINDOW_NORMAL)  # Make the window resizable
        cv2.resizeWindow('Frame', 640, 480)
        cv2.imshow('Frame', frame)
        cv2.putText(frame, f'Frame: {current_frame}/{frame_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        key = cv2.waitKey(0)
        
        # Save on S
        if key == ord('s'):
            d = video_file.rsplit(sep='\\', maxsplit = 1)[0] + f'\\Frames\\frame_{current_frame}.jpg'
            cv2.imwrite(d, frame)
            print(f"Frame {current_frame} saved at {d}")
            current_frame += skip_frames
        # Move to the next frame on D
        elif key == ord('a'):
            current_frame += skip_frames

        # Exit on Q
        elif key == ord('q'):
            break

        cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
    cap.release()
    cv2.destroyAllWindows()

### Cycle through frames of selected videos and choose those in which all internal corners of chessboard (or aruco markers) are visible

Change the names of the saved video files and go through each one by one. Save several frames with full checkerboards in focus and at several angles. Optionally give the number of frames between viewed frames (skip_frames) and the first frame (start_frame). If it exists, selected frames are saved in the 'Frames' folder in the same directory as the movie.

In [ ]:
vidfile = r'path\to\calibration\video.avi'
select_frames(vidfile, skip_frames=1, start_frame=0)

# Calibrate using these frames

Frames from individual cameras should be placed in dedicated folders (Camera1...Camera2 etc) as seen in the cell below

In [ ]:
chessboard_root = r"path\to\video\directory\"
######################################################################################
data_folder_camera1 = chessboard_root + r"Camera1\Frames\\"
data_folder_camera2 = chessboard_root + r"Camera2\Frames\\"
data_folder_camera3 = chessboard_root + r"Camera3\Frames\\"
######################################################################################

datafolders_chess = [data_folder_camera1,data_folder_camera2,data_folder_camera3]

## Compute intrinsic parameters of camera from chessboard patterns

In [ ]:
def ChessboardCalibrate(datafolder, patternsize = (14, 19), report = True, save = False):
    """
    Calibrates a camera using images of a chessboard pattern.

    Args:
        datafolder (str): The folder path containing images of the chessboard pattern.
        patternsize (tuple, optional): The dimensions of the chessboard pattern (rows, columns). Defaults to (14, 19).
        report (bool, optional): Whether to print calibration report. Defaults to True.
        save (bool, optional): Whether to save calibration data. Defaults to False.

    Returns:
        tuple: A tuple containing:
            - reproj_error (float): Reprojection error of the calibration.
            - intrinsics (numpy.ndarray): Camera intrinsic matrix.
            - distortion (numpy.ndarray): Distortion coefficients.
            - rvecs (list): Rotation vectors for each calibration image.
            - tvecs (list): Translation vectors for each calibration image.
            - imagePoints (list): Image coordinates of detected corners.
            - objectPoints (list): Object coordinates of detected corners.
            - fnames_chessboard_found (list): File names of images where the chessboard pattern was found (for undistorting).

    Calibrates a camera using images of a chessboard pattern. 
    Each image in the specified data folder is processed to detect the chessboard corners. 
    If the chessboard pattern is found, corresponding object points (3D) and image points (2D) are stored.
    The camera is then calibrated using the collected points.

    If 'report' is set to True, a calibration report including the reprojection error and principal points is printed.
    If 'save' is set to True, the calibration data is saved to a YAML file named 'Calibration.yml' in the datafolder.

    Note:
        This function uses OpenCV for camera calibration and image processing.
"""

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    pattern = np.zeros((1, patternsize[0] * patternsize[1], 3), np.float32)
    pattern[0,:,:2] = np.mgrid[0:patternsize[0], 0:patternsize[1]].T.reshape(-1, 2) * 0.001 # size of pattern in real world (m) to put objpoints into correct scale
    fnames = glob.glob(datafolder + '/*.jpg')
    fnames_chessboard_found = []
    objectPoints = []
    imagePoints = []

    for fname in fnames:
        if report:
            print("Processing {0}...".format(os.path.split(fname)[-1]), end=" ")
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        flags = cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE
        ret, corners = cv2.findChessboardCorners(gray, patternsize, flags)
        if ret:
            if report:
                print("Success")  
            fnames_chessboard_found.append(fname)
        else: 
            if report:
                print("Not found!")
        
        if ret:
            corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            objectPoints.append(pattern)
            imagePoints.append(corners)
            
            img = cv2.drawChessboardCorners(img, patternsize, corners, ret)
        if report:
            plt.figure()
            plt.imshow(img)
            plt.draw()
    if report:
        print("Calibrating camera with {0} points from {1} images...".format(
            np.prod(np.array(imagePoints).shape) // 2,
            len(imagePoints)))
    reproj_error, intrinsics, distortion, rvecs, tvecs = cv2.calibrateCamera(objectPoints, imagePoints, gray.shape[::-1], None, None)

    if report:
        print(f"Reprojection error: {reproj_error}")
        print(f'Principle points: ({intrinsics[0,2]} / {img.shape[1]/2}) : {intrinsics[1,2]} / {img.shape[0]/2}')

    if save:
        sname = "{0}\\Calibration_int.yml".format(os.path.split(fname)[0])
        print("Writing calibration file {0}...".format(sname))
        flags = cv2.FILE_STORAGE_FORMAT_YAML + cv2.FILE_STORAGE_WRITE
        s = cv2.FileStorage(sname,flags)
        s.write('image_width', img.shape[1])
        s.write('image_height', img.shape[0])
        s.write('camera_matrix', intrinsics)
        s.write('distortion_coefficients', distortion)
        s.write('reprojection_error', reproj_error)
        s.release()
        

    return reproj_error, intrinsics, distortion, rvecs, tvecs, imagePoints, objectPoints, fnames_chessboard_found
    

In [ ]:
intrinsics_chess, distortions_chess = [], []
for datafolder in datafolders_chess:
    reproj_error, intrinsics, distortion,_, _, imagePoints, objectPoints, fnames = ChessboardCalibrate(datafolder, report = False, save = False)
    intrinsics_chess.append(intrinsics)
    # distortions_chess.append(np.array([0,0,0,0,0],dtype=float))
    distortions_chess.append(distortion)

### Now we will perform pose estimation for a set of Ch-Aruco or Aruco marker pattern

Define aruco board

In [ ]:
# Set ARUCO marker parameters
ARUCO_DICT = aruco.DICT_4X4_1000   # Dictionary ID
# ARUCO_DICT = aruco.DICT_ARUCO_ORIGINAL
SQUARES_VERTICALLY = 20             # Number of squares vertically
SQUARES_HORIZONTALLY = 15             # Number of squares horizontally
SQUARE_LENGTH = 10                   # Square side length (in pixels)
MARKER_LENGTH = 7                   # ArUco marker side length (in pixels)
MARGIN_PX = 20                       # Margins size (in pixels)                    # Margins size (in pixels)
dictionary = cv2.aruco.getPredefinedDictionary(ARUCO_DICT)
board = cv2.aruco.CharucoBoard((SQUARES_VERTICALLY, SQUARES_HORIZONTALLY), SQUARE_LENGTH, MARKER_LENGTH, dictionary)

### Define path to charuco marker frames

In [ ]:
reproj_root = r'path\to\charuco\ directory\'
reproj_folders = [reproj_root + r'Camera1\Frames\\',
                  reproj_root + r'Camera2\Frames\\',
                  reproj_root + r'Camera3\Frames\\']

In [ ]:
# Load calibration data (intrinsics) of the cameras
camera_matrix = intrinsics_chess  # Intrinsics matrix (3x3) for each camera
dist_coeffs = distortions_chess    # Distortion coefficients for each camera

# Create Charuco board
charuco_dict = dictionary
charuco_board = board

# Iterate through images captured by each camera

for camera,datafolder in enumerate(reproj_folders):  # Assuming you have 3 cameras
    image_file = [os.path.join(datafolder, f) for f in os.listdir(datafolder) if f.endswith(".jpg")][0]
    # Load image from the camera
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect markers and corners
    corners, ids, _ = aruco.detectMarkers(image, charuco_dict)
    _, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco(
        corners, ids, image, charuco_board)

    if charuco_corners is not None and charuco_ids is not None:
        # Estimate pose (extrinsics) of the camera
        retval, rvec, tvec = aruco.estimatePoseCharucoBoard(
            charuco_corners, charuco_ids, charuco_board, camera_matrix[camera], dist_coeffs[camera],np.empty(1),np.empty(1))

        if retval:
            # Draw axis on the image to visualize pose
            cv2.drawFrameAxes(image, camera_matrix[camera], dist_coeffs[camera], rvec, tvec, 0.1)

            # Print rotation and translation vectors
            print(f"Camera {camera} - Rotation Vector:\n{rvec}")
            print(f"Camera {camera} - Translation Vector:\n{tvec}")

            # Display the image with axes
            cv2.imshow(f'Camera {camera} - Pose', image)
            cv2.waitKey(0)
        if save:
            sname = "{0}\\Calibration_ext.yml".format(os.path.split(fname)[0])
            print("Writing calibration file {0}...".format(sname))
            flags = cv2.FILE_STORAGE_FORMAT_YAML + cv2.FILE_STORAGE_WRITE
            s = cv2.FileStorage(sname,flags)
            s.write('image_width', img.shape[1])
            s.write('image_height', img.shape[0])
            s.write('camera_matrix', intrinsics)
            s.write('distortion_coefficients', distortion)
            s.write('reprojection_error', reproj_error)
            s.release()
# Close all OpenCV windows
cv2.destroyAllWindows()

In [ ]:
if save:
        sname = f"{(os.path.split(fname)[0])}\\Camera{camera}.yml"
        print("Writing calibration file {0}...".format(sname))
        flags = cv2.FILE_STORAGE_FORMAT_YAML + cv2.FILE_STORAGE_WRITE
        s = cv2.FileStorage(sname,flags)
        s.write('image_width', img.shape[1])
        s.write('image_height', img.shape[0])
        s.write('camera_matrix', intrinsics)
        s.write('distortion_coefficients', distortion)
        s.write('reprojection_error', reproj_error)
        s.release()